In [167]:
import numpy as np
import sys, os, shutil
import pandas as pd
import collections as cl
import itertools as it
import timeit
import pandas as pd
import datetime as dt
import matplotlib.pyplot as plt
import matplotlib
import seaborn as sns
import copy as cp
import pyprojroot as pr
import random as rand

In [13]:
User=['Emory','WG'][0]
print(User)

Emory


In [15]:
#Change directory
if User=='Emory':
    os.chdir(pr.here("./Data/fake"))
elif User == 'WG':
    os.chdir(pr.here("./Data/real"))
os.getcwd()

'C:\\Users\\yche465\\Desktop\\AIM 1\\Codes\\PrEP-Traj-Clustering\\Data\\fake'

In [40]:
#Import original prescription date records
prescrip_dates_orig=pd.read_csv("syndata_13k.csv").astype({'ID':'int'})
#convert the column's data type to date time 
prescrip_dates_orig['PrEP_Start']= pd.to_datetime(prescrip_dates_orig['PrEP_Start'])
prescrip_dates_orig['PrEP_End']= pd.to_datetime(prescrip_dates_orig['PrEP_End'])

In [64]:
#Import sampled patient ID
Sampled_SeroProtect = np.loadtxt('SeroProtect_4k.txt', usecols=range(104), delimiter=",")
Sampled_ID=Sampled_SeroProtect[:,0].astype('int32')

In [65]:
#Extract prescription record for sampled pateints
prescrip_dates_sampled=pd.DataFrame(prescrip_dates_orig.loc[prescrip_dates_orig['ID'].isin(Sampled_ID)])

In [107]:
#Create a column for PrEP assessment interval: 
## the time interval bounded by two consecutive PrEP prescription fill dates, 
## or the end date of the two-year tracking period 

startdate=prescrip_dates_sampled.groupby("ID").agg('min').drop(columns=['PrEP_End'])
startdate=startdate.rename(columns={'PrEP_Start':'Initiation'})
prescrip_dates_sampled2=prescrip_dates_sampled.merge(startdate, on="ID",how='left')
prescrip_dates_sampled2['Two_Year_Mark']=prescrip_dates_sampled2['Initiation']+dt.timedelta(days=365*2)

In [125]:
within_bound=prescrip_dates_sampled2['Two_Year_Mark']-prescrip_dates_sampled2['PrEP_Start']<=dt.timedelta(days=365*2)
prescrip_dates_sampled3=prescrip_dates_sampled2.loc[within_bound,:]
#no rows were out of the 2-year bound
print(prescrip_dates_sampled3.shape==prescrip_dates_sampled2.shape)

#create a column indicating next PrEP start date per row
prescrip_dates_sampled3['Next_PrEP_Start']=prescrip_dates_sampled3.groupby('ID').shift(-1)['PrEP_Start']

#create a column for the end date of PrEP assessment interval
prescrip_dates_sampled3['PrEP_assessment_interval']=np.where(pd.isnull(prescrip_dates_sampled3['Next_PrEP_Start']),
                                                            prescrip_dates_sampled3['Two_Year_Mark'],
                                                            prescrip_dates_sampled3['Next_PrEP_Start'])
#create a column for the end date of PrEP consumption interval
prescrip_dates_sampled3['PrEP_consumption_interval']=np.where(prescrip_dates_sampled3['PrEP_assessment_interval']>prescrip_dates_sampled3['PrEP_End']+dt.timedelta(days=7),
                                                             prescrip_dates_sampled3['PrEP_End']+dt.timedelta(days=7),
                                                             prescrip_dates_sampled3['PrEP_assessment_interval'])

True


In [129]:
#extract ID for those eligible for PrEP nondaily dosing regimen
prescrip_dates_sampled3['ND_eligible']=prescrip_dates_sampled3['PrEP_consumption_interval']<prescrip_dates_sampled3['PrEP_assessment_interval']
DAT=prescrip_dates_sampled3.groupby('ID').agg('max')
ND_Elig_ID=np.array(DAT.index.values.tolist())[DAT['ND_eligible'].tolist()]

,PrEP_Start,PrEP_End,Initiation,Two_Year_Mark,Next_PrEP_Start,PrEP_assessment_interval,PrEP_consumption_interval,ND_eligible
ID,,,,,,,,
206,2024-01-04,2024-02-24,2023-06-26,2025-06-25,2024-01-04,2025-06-25,2024-02-29,True
207,2023-09-06,2023-12-12,2023-09-06,2025-09-05,NaT,2025-09-05,2023-12-17,True
265,2023-04-05,2023-08-17,2023-04-05,2025-04-04,NaT,2025-04-04,2023-08-22,True
442,2023-09-21,2024-05-28,2023-01-14,2025-01-13,2023-09-21,2025-01-13,2024-06-02,True
484,2023-09-27,2023-12-26,2023-09-27,2025-09-26,NaT,2025-09-26,2023-12-31,True
...,...,...,...,...,...,...,...,...
191726,2023-08-18,2023-11-17,2023-08-18,2025-08-17,NaT,2025-08-17,2023-11-22,True
191794,2023-02-18,2023-06-19,2023-02-18,2025-02-17,NaT,2025-02-17,2023-06-24,True
191800,2023-09-12,2024-02-13,2023-09-12,2025-09-11,NaT,2025-09-11,2024-02-18,True


In [205]:
#Select 10% of PrEP users to follow nondaily dosing regimen
##Non-daily PrEP users will be randomly selected from the subset of PrEP users who have >=1 PrEP assessment intervals
##in which the interval lengths are greater than the length of the adherent PrEP consumption periods
rand.seed(30)
ID_NDUsers=sorted(rand.sample(sorted(ND_Elig_ID),int(len(Sampled_ID)*0.1)))

In [206]:
#4 doses per week regimen: they would use the non-daily PrEP regimen for the PrEP assessment intervals 
# in which the interval lengths were greater than those of the adherent PrEP consumption periods.

[206,
 574,
 745,
 817,
 888,
 1195,
 1332,
 1835,
 2299,
 3116,
 3736,
 3781,
 3898,
 4138,
 4279,
 4419,
 4488,
 4681,
 5282,
 5293,
 5466,
 5960,
 6574,
 7617,
 7835,
 8653,
 12106,
 12171,
 12631,
 12712,
 12735,
 13090,
 13472,
 13541,
 13661,
 14021,
 14854,
 15005,
 17215,
 17860,
 18040,
 18593,
 19537,
 19787,
 20251,
 20357,
 20485,
 21306,
 21349,
 21497,
 21671,
 21708,
 21759,
 22342,
 22346,
 22458,
 22557,
 23030,
 24203,
 24921,
 25013,
 25747,
 26188,
 26293,
 26341,
 27624,
 27780,
 27811,
 27957,
 29008,
 30722,
 30761,
 31057,
 31913,
 32233,
 33150,
 33569,
 33755,
 33889,
 34684,
 34983,
 35027,
 35717,
 35930,
 36127,
 36329,
 36694,
 36874,
 36964,
 37005,
 37229,
 37351,
 37554,
 38824,
 39168,
 40175,
 40642,
 41445,
 43063,
 43803,
 44242,
 46155,
 47412,
 47422,
 47572,
 48774,
 48852,
 49315,
 49525,
 49954,
 50052,
 50289,
 51087,
 51945,
 52317,
 53768,
 54483,
 54688,
 54941,
 55241,
 56016,
 56550,
 57610,
 57845,
 57885,
 58481,
 59235,
 60031,
 60103,

True

In [98]:
prescrip_dates_sampled3

,ID,PrEP_Start,PrEP_End,Initiation,Two_Year_Mark
0,206,2023-06-26,2023-10-17,2023-06-26,2025-06-25
1,206,2024-01-04,2024-02-24,2023-06-26,2025-06-25
2,207,2023-09-06,2023-12-12,2023-09-06,2025-09-05
3,265,2023-04-05,2023-08-17,2023-04-05,2025-04-04
4,442,2023-01-14,2023-06-09,2023-01-14,2025-01-13
...,...,...,...,...,...
6048,191726,2023-08-18,2023-11-17,2023-08-18,2025-08-17
6049,191794,2023-02-18,2023-06-19,2023-02-18,2025-02-17
6050,191800,2023-09-12,2024-02-13,2023-09-12,2025-09-11
6051,191819,2023-12-18,2024-03-04,2023-12-18,2025-12-17


In [100]:
prescrip_dates_sampled2.shape

(6053, 5)